In [1]:
import requests
from requests.exceptions import ConnectionError
import lxml.html as lh
import pandas as pd
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [2]:
companies1 = pd.read_csv('List_of_companies.csv', encoding = "ISO-8859-1")
companies = pd.read_csv('List_of_companies.csv', encoding = "ISO-8859-1")

print(companies.dtypes)

companies['Security'] = companies['Security'].str.replace(u'\xa0', u' ')#this was messing with the classes
companies['Security'] = companies['Security'].str.replace(r' \(Class A\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r' \(Class B\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r' \(Class C\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r'\(', '')#need to remove parenthesis too for the url
companies['Security'] = companies['Security'].str.replace(r'\)', '')
companies['Security'] = companies['Security'].str.replace(r'\.com', '-com')#for amazon and salesforce
companies['Security'] = companies['Security'].str.replace(r'Corporation', 'corp')#corporation got shortened to corp
companies['Security'] = companies['Security'].str.replace(r'corporation', 'corp')
companies['Security'] = companies['Security'].str.replace(r'Company', 'co')#company got shortened to co
companies['Security'] = companies['Security'].str.replace(r'company', 'co')
companies['Security'] = companies['Security'].str.replace(r'&', 'and')#& is and for some reason
companies['Security'] = companies['Security'].str.replace(r' ', '-')#the url uses hyphens a lot
companies['Security'] = companies['Security'].str.replace(r'.', '')#don't want these here
companies['Security'] = companies['Security'].str.replace(r',', '')
companies['Security'] = companies['Security'].str.replace(r'!', '')
companies['Security'] = companies['Security'].str.replace(r'\'', '')

companies['Security'] = companies['Security'].str.lower()#lowercases everything
print(len(companies))
companies

Symbol                   object
Security                 object
SEC filings              object
GICS Sector              object
GICS Sub-Industry        object
Headquarters Location    object
Date first added         object
CIK                       int64
Founded                  object
dtype: object
500


,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3m-co,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",8/9/1976,66740,1902
1,ABT,abbott-laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/1964,1800,1888
2,ABBV,abbvie-inc,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012,1551152,2013 (1888)
3,ABMD,abiomed-inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/2018,815094,1981
4,ACN,accenture-plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/2011,1467373,1989
5,ATVI,activision-blizzard,reports,Communication Services,Interactive Home Entertainment,"Santa Monica, California",8/31/2015,718877,2008
6,ADBE,adobe-inc,reports,Information Technology,Application Software,"San Jose, California",5/5/1997,796343,1982
7,AMD,advanced-micro-devices-inc,reports,Information Technology,Semiconductors,"Santa Clara, California",3/20/2017,2488,1969
8,AAP,advance-auto-parts,reports,Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina",7/9/2015,1158449,1932
9,AES,aes-corp,reports,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",10/2/1998,874761,1981


In [3]:
#getting url this is for testing purposes
for n in range(10):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-Executive-Salaries.html'
    print(url)


https://www1.salary.com/3m-co-Executive-Salaries.html
https://www1.salary.com/abbott-laboratories-Executive-Salaries.html
https://www1.salary.com/abbvie-inc-Executive-Salaries.html
https://www1.salary.com/abiomed-inc-Executive-Salaries.html
https://www1.salary.com/accenture-plc-Executive-Salaries.html
https://www1.salary.com/activision-blizzard-Executive-Salaries.html
https://www1.salary.com/adobe-inc-Executive-Salaries.html
https://www1.salary.com/advanced-micro-devices-inc-Executive-Salaries.html
https://www1.salary.com/advance-auto-parts-Executive-Salaries.html
https://www1.salary.com/aes-corp-Executive-Salaries.html


In [4]:
#utilities
def getUrl(n, mod = ''):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + mod + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getIncUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-inc' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getCoUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-co' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getCorpUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-corp' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getPlcUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-plc' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getContent(page):
    doc=lh.fromstring(page.content)
    tr_elements=doc.xpath('//tr')
    return tr_elements

In [5]:
def failureCase(col, good, alt_tr_elements):
    for j in range(0,len(alt_tr_elements)):
        #T is our j'th row
        T=alt_tr_elements[j]

        #Only check rows of size 4 (with CEO Name, CEO Pay, Median Employee Pay,
        #and CEO Pay Ratio)
        if len(T)==4:
            #checks if the current row has the 4 maagic words
            table_Name = ['','','','']
            i=0
            for t in T:
                name = t.text_content()
                name = name.__str__()
                table_Name[i]=name
                i+=1

            #if it does, I'll take the data from the row right below it
            if (table_Name[0] == 'CEO Name' and table_Name[1] == 'CEO Pay' and table_Name[2] == 'Median Employee Pay' and table_Name[3] == 'CEO Pay Ratio'):
                T=alt_tr_elements[j+1]
                #i is the index of our column
                i=0

                #this is a temporary storage of the content of each line of the csv
                content = []

                #Iterate through each element of the row
                for t in T.iterchildren():
                    #this part if for cleaning the cell contents
                    data=t.text_content() #gets the content of cell
                    data = data.__str__() #turns content to string
                    data = data.replace(r'$', '') #takes out dollar sign for number conversion
                    data = data.replace(r',', '') #takes out comma for number conversion
                    data = data.strip() #clears our \r and \n and spaces, cleans up the content string
                    data = data.replace(r'CEO Pay', '') #cleans up the content string
                    data = data.replace(r'Median Employee Pay', '') #cleans up the content string
                    data = data.replace(r'CEO Pay Ratio', '') #cleans up the content string
                    data = data.replace(r'Ratio', '') #cleans up the content string
                    data = data.replace(r':1', '') #takes out :1 in the ratio for number conversion
                    data = data.strip() #clears our \r and \n and spaces again to finish cleaning up

                    content.append(data)

                    #Check if row is empty
                    if i>0:
                    #Convert any numerical value to integers
                        try:
                            data=int(data)
                        except:
                            pass
                    #Append the data to col
                    col[i + 1][1].append(data)
                    #Increment i for the next column
                    i+=1
                #col[0][1].append(currName1)
                return True

        return False

In [6]:
#Makes a list of to make the graph later on
col=[('Company Name', []), ('CEO Name', []), ('CEO Pay', []), ('Median Employee Pay', []), ('CEO Pay Ratio', [])]

#goes through all the webpages
for n in range(len(companies)):
    currName = companies.at[n, 'Security']
    currName1 =companies1.at[n, 'Security']
    col[0][1].append(currName1)
    #getting url
    url='https://www1.salary.com/' + currName + '-Executive-Salaries.html'
    page=requests.get(url)
    #getting all the tables
    tr_elements=getContent(page)
    #indicates that we failed to find the magics on this particular page
    fail = True
    #skip = False
    """try:
            doc=lh.fromstring(page.content)
            tr_elements=doc.xpath('//tr')
        except:
            skip = True"""
    """if skip:
            print(1)
            print(currName1)
            print(currName)
            print(url)
            col[1][1].append('')
            col[2][1].append(-1)
            col[3][1].append(-1)
            col[4][1].append(-1)"""
    #this will find the table that has the relevant info for the CEO,and put the correct info into col for this one page
    for j in range(0,len(tr_elements)):
        #T is our j'th row
        T=tr_elements[j]

        #Only check rows of size 4 (with CEO Name, CEO Pay, Median Employee Pay,
        #and CEO Pay Ratio)
        if len(T)==4:
            #checks if the current row has the 4 maagic words
            table_Name = ['','','','']
            i=0
            for t in T:
                name = t.text_content()
                name = name.__str__()
                table_Name[i]=name
                i+=1

            #if it does, I'll take the data from the row right below it
            if (table_Name[0] == 'CEO Name' and table_Name[1] == 'CEO Pay' and table_Name[2] == 'Median Employee Pay' and table_Name[3] == 'CEO Pay Ratio'):
                #setting fail to false
                fail = False
                T=tr_elements[j+1]
                #i is the index of our column
                i=0

                #this is a temporary storage of the content of each line of the csv
                content = []

                #Iterate through each element of the row
                for t in T.iterchildren():
                    #this part if for cleaning the cell contents
                    data=t.text_content() #gets the content of cell
                    data = data.__str__() #turns content to string
                    data = data.replace(r'$', '') #takes out dollar sign for number conversion
                    data = data.replace(r',', '') #takes out comma for number conversion
                    data = data.strip() #clears our \r and \n and spaces, cleans up the content string
                    data = data.replace(r'CEO Pay', '') #cleans up the content string
                    data = data.replace(r'Median Employee Pay', '') #cleans up the content string
                    data = data.replace(r'CEO Pay Ratio', '') #cleans up the content string
                    data = data.replace(r'Ratio', '') #cleans up the content string
                    data = data.replace(r':1', '') #takes out :1 in the ratio for number conversion
                    data = data.strip() #clears our \r and \n and spaces again to finish cleaning up

                    content.append(data)

                    #Check if row is empty
                    if i>0:
                    #Convert any numerical value to integers
                        try:
                            data=int(data)
                        except:
                            pass
                    #Append the data to col
                    col[i + 1][1].append(data)
                    #Increment i for the next column
                    i+=1
                #col[0][1].append(currName1)
                break
    
    if fail:
        print(currName1)
        print(currName)
        print(url)
        #put these into the get url
        alts = ['-inc','-co','-corp','-plc','-new']
        #good indicated whether or not the new url works
        good = False
        for i in range(len(alts)):
            url='https://www1.salary.com/' + currName + alts[i] + '-Executive-Salaries.html'
            page=requests.get(url)
            alt_tr_elements=getContent(page)
            for j in range(0,len(alt_tr_elements)):
                #T is our j'th row
                T=alt_tr_elements[j]

                #Only check rows of size 4 (with CEO Name, CEO Pay, Median Employee Pay,
                #and CEO Pay Ratio)
                if len(T)==4:
                    #checks if the current row has the 4 maagic words
                    table_Name = ['','','','']
                    i=0
                    for t in T:
                        name = t.text_content()
                        name = name.__str__()
                        table_Name[i]=name
                        i+=1

                    #if it does, I'll take the data from the row right below it
                    if (table_Name[0] == 'CEO Name' and table_Name[1] == 'CEO Pay' and table_Name[2] == 'Median Employee Pay' and table_Name[3] == 'CEO Pay Ratio'):
                        T=alt_tr_elements[j+1]
                        good = True
                        #i is the index of our column
                        i=0

                        #this is a temporary storage of the content of each line of the csv
                        content = []

                        #Iterate through each element of the row
                        for t in T.iterchildren():
                            #this part if for cleaning the cell contents
                            data=t.text_content() #gets the content of cell
                            data = data.__str__() #turns content to string
                            data = data.replace(r'$', '') #takes out dollar sign for number conversion
                            data = data.replace(r',', '') #takes out comma for number conversion
                            data = data.strip() #clears our \r and \n and spaces, cleans up the content string
                            data = data.replace(r'CEO Pay', '') #cleans up the content string
                            data = data.replace(r'Median Employee Pay', '') #cleans up the content string
                            data = data.replace(r'CEO Pay Ratio', '') #cleans up the content string
                            data = data.replace(r'Ratio', '') #cleans up the content string
                            data = data.replace(r':1', '') #takes out :1 in the ratio for number conversion
                            data = data.strip() #clears our \r and \n and spaces again to finish cleaning up

                            content.append(data)

                            #Check if row is empty
                            if i>0:
                            #Convert any numerical value to integers
                                try:
                                    data=int(data)
                                except:
                                    pass
                            #Append the data to col
                            col[i + 1][1].append(data)
                            #Increment i for the next column
                            i+=1
                        #col[0][1].append(currName1)
            if good:
                break
            else:
                print(url)
        #this means the page failed
        if (good == False):
            col[1][1].append('na')
            col[2][1].append(-1)
            col[3][1].append(-1)
            col[4][1].append(-1)

Abbott Laboratories
abbott-laboratories
https://www1.salary.com/abbott-laboratories-Executive-Salaries.html
https://www1.salary.com/abbott-laboratories-inc-Executive-Salaries.html
https://www1.salary.com/abbott-laboratories-co-Executive-Salaries.html
https://www1.salary.com/abbott-laboratories-corp-Executive-Salaries.html
https://www1.salary.com/abbott-laboratories-plc-Executive-Salaries.html
https://www1.salary.com/abbott-laboratories-new-Executive-Salaries.html
Accenture plc
accenture-plc
https://www1.salary.com/accenture-plc-Executive-Salaries.html
https://www1.salary.com/accenture-plc-inc-Executive-Salaries.html
https://www1.salary.com/accenture-plc-co-Executive-Salaries.html
https://www1.salary.com/accenture-plc-corp-Executive-Salaries.html
https://www1.salary.com/accenture-plc-plc-Executive-Salaries.html
https://www1.salary.com/accenture-plc-new-Executive-Salaries.html
Activision Blizzard
activision-blizzard
https://www1.salary.com/activision-blizzard-Executive-Salaries.html
Adva

C. H. Robinson Worldwide
c-h-robinson-worldwide
https://www1.salary.com/c-h-robinson-worldwide-Executive-Salaries.html
Cabot Oil & Gas
cabot-oil-and-gas
https://www1.salary.com/cabot-oil-and-gas-Executive-Salaries.html
https://www1.salary.com/cabot-oil-and-gas-inc-Executive-Salaries.html
https://www1.salary.com/cabot-oil-and-gas-co-Executive-Salaries.html
Cadence Design Systems
cadence-design-systems
https://www1.salary.com/cadence-design-systems-Executive-Salaries.html
Campbell Soup
campbell-soup
https://www1.salary.com/campbell-soup-Executive-Salaries.html
https://www1.salary.com/campbell-soup-inc-Executive-Salaries.html
Capital One Financial
capital-one-financial
https://www1.salary.com/capital-one-financial-Executive-Salaries.html
https://www1.salary.com/capital-one-financial-inc-Executive-Salaries.html
https://www1.salary.com/capital-one-financial-co-Executive-Salaries.html
Carnival Corp.
carnival-corp
https://www1.salary.com/carnival-corp-Executive-Salaries.html
https://www1.sala

https://www1.salary.com/discover-financial-services-co-Executive-Salaries.html
https://www1.salary.com/discover-financial-services-corp-Executive-Salaries.html
https://www1.salary.com/discover-financial-services-plc-Executive-Salaries.html
https://www1.salary.com/discover-financial-services-new-Executive-Salaries.html
Dish Network
dish-network
https://www1.salary.com/dish-network-Executive-Salaries.html
https://www1.salary.com/dish-network-inc-Executive-Salaries.html
https://www1.salary.com/dish-network-co-Executive-Salaries.html
Dollar General
dollar-general
https://www1.salary.com/dollar-general-Executive-Salaries.html
https://www1.salary.com/dollar-general-inc-Executive-Salaries.html
https://www1.salary.com/dollar-general-co-Executive-Salaries.html
Dollar Tree
dollar-tree
https://www1.salary.com/dollar-tree-Executive-Salaries.html
Dominion Energy
dominion-energy
https://www1.salary.com/dominion-energy-Executive-Salaries.html
Domino's Pizza
dominos-pizza
https://www1.salary.com/domin

https://www1.salary.com/globe-life-inc-plc-Executive-Salaries.html
https://www1.salary.com/globe-life-inc-new-Executive-Salaries.html
Goldman Sachs Group
goldman-sachs-group
https://www1.salary.com/goldman-sachs-group-Executive-Salaries.html
Grainger (W.W.) Inc.
grainger-ww-inc
https://www1.salary.com/grainger-ww-inc-Executive-Salaries.html
https://www1.salary.com/grainger-ww-inc-inc-Executive-Salaries.html
https://www1.salary.com/grainger-ww-inc-co-Executive-Salaries.html
https://www1.salary.com/grainger-ww-inc-corp-Executive-Salaries.html
https://www1.salary.com/grainger-ww-inc-plc-Executive-Salaries.html
https://www1.salary.com/grainger-ww-inc-new-Executive-Salaries.html
Hartford Financial Svc.Gp.
hartford-financial-svcgp
https://www1.salary.com/hartford-financial-svcgp-Executive-Salaries.html
https://www1.salary.com/hartford-financial-svcgp-inc-Executive-Salaries.html
https://www1.salary.com/hartford-financial-svcgp-co-Executive-Salaries.html
https://www1.salary.com/hartford-financ

https://www1.salary.com/iron-mountain-incorporated-plc-Executive-Salaries.html
https://www1.salary.com/iron-mountain-incorporated-new-Executive-Salaries.html
Jack Henry & Associates
jack-henry-and-associates
https://www1.salary.com/jack-henry-and-associates-Executive-Salaries.html
https://www1.salary.com/jack-henry-and-associates-inc-Executive-Salaries.html
https://www1.salary.com/jack-henry-and-associates-co-Executive-Salaries.html
https://www1.salary.com/jack-henry-and-associates-corp-Executive-Salaries.html
https://www1.salary.com/jack-henry-and-associates-plc-Executive-Salaries.html
https://www1.salary.com/jack-henry-and-associates-new-Executive-Salaries.html
Jacobs Engineering Group
jacobs-engineering-group
https://www1.salary.com/jacobs-engineering-group-Executive-Salaries.html
J. B. Hunt Transport Services
j-b-hunt-transport-services
https://www1.salary.com/j-b-hunt-transport-services-Executive-Salaries.html
https://www1.salary.com/j-b-hunt-transport-services-inc-Executive-Salar

https://www1.salary.com/mid-america-apartments-inc-Executive-Salaries.html
https://www1.salary.com/mid-america-apartments-co-Executive-Salaries.html
https://www1.salary.com/mid-america-apartments-corp-Executive-Salaries.html
https://www1.salary.com/mid-america-apartments-plc-Executive-Salaries.html
https://www1.salary.com/mid-america-apartments-new-Executive-Salaries.html
Mohawk Industries
mohawk-industries
https://www1.salary.com/mohawk-industries-Executive-Salaries.html
Molson Coors Beverage Company
molson-coors-beverage-co
https://www1.salary.com/molson-coors-beverage-co-Executive-Salaries.html
https://www1.salary.com/molson-coors-beverage-co-inc-Executive-Salaries.html
https://www1.salary.com/molson-coors-beverage-co-co-Executive-Salaries.html
https://www1.salary.com/molson-coors-beverage-co-corp-Executive-Salaries.html
https://www1.salary.com/molson-coors-beverage-co-plc-Executive-Salaries.html
https://www1.salary.com/molson-coors-beverage-co-new-Executive-Salaries.html
Mondelez I

https://www1.salary.com/pinnacle-west-capital-inc-Executive-Salaries.html
https://www1.salary.com/pinnacle-west-capital-co-Executive-Salaries.html
Pioneer Natural Resources
pioneer-natural-resources
https://www1.salary.com/pioneer-natural-resources-Executive-Salaries.html
https://www1.salary.com/pioneer-natural-resources-inc-Executive-Salaries.html
https://www1.salary.com/pioneer-natural-resources-co-Executive-Salaries.html
https://www1.salary.com/pioneer-natural-resources-corp-Executive-Salaries.html
https://www1.salary.com/pioneer-natural-resources-plc-Executive-Salaries.html
https://www1.salary.com/pioneer-natural-resources-new-Executive-Salaries.html
PNC Financial Services
pnc-financial-services
https://www1.salary.com/pnc-financial-services-Executive-Salaries.html
https://www1.salary.com/pnc-financial-services-inc-Executive-Salaries.html
https://www1.salary.com/pnc-financial-services-co-Executive-Salaries.html
https://www1.salary.com/pnc-financial-services-corp-Executive-Salaries.

https://www1.salary.com/sl-green-realty-inc-Executive-Salaries.html
https://www1.salary.com/sl-green-realty-co-Executive-Salaries.html
Snap-on
snap-on
https://www1.salary.com/snap-on-Executive-Salaries.html
Stanley Black & Decker
stanley-black-and-decker
https://www1.salary.com/stanley-black-and-decker-Executive-Salaries.html
SVB Financial
svb-financial
https://www1.salary.com/svb-financial-Executive-Salaries.html
https://www1.salary.com/svb-financial-inc-Executive-Salaries.html
https://www1.salary.com/svb-financial-co-Executive-Salaries.html
https://www1.salary.com/svb-financial-corp-Executive-Salaries.html
https://www1.salary.com/svb-financial-plc-Executive-Salaries.html
https://www1.salary.com/svb-financial-new-Executive-Salaries.html
Synopsys Inc.
synopsys-inc
https://www1.salary.com/synopsys-inc-Executive-Salaries.html
https://www1.salary.com/synopsys-inc-inc-Executive-Salaries.html
https://www1.salary.com/synopsys-inc-co-Executive-Salaries.html
https://www1.salary.com/synopsys-in

ConnectionError: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))

In [7]:
col

[('Company Name',
  ['3M Company',
   'Abbott Laboratories',
   'AbbVie Inc.',
   'ABIOMED Inc',
   'Accenture plc',
   'Activision Blizzard',
   'Adobe Inc.',
   'Advanced Micro Devices Inc',
   'Advance Auto Parts',
   'AES Corp',
   'AFLAC Inc',
   'Agilent Technologies Inc',
   'Air Products & Chemicals Inc',
   'Akamai Technologies Inc',
   'Alaska Air Group Inc',
   'Albemarle Corp',
   'Alexandria Real Estate Equities',
   'Alexion Pharmaceuticals',
   'Align Technology',
   'Allegion',
   'Alliant Energy Corp',
   'Allstate Corp',
   'Alphabet Inc.',
   'Altria Group Inc',
   'Amazon.com Inc.',
   'Amcor plc',
   'Ameren Corp',
   'American Airlines Group',
   'American Electric Power',
   'American Express Co',
   'American International Group',
   'American Tower Corp.',
   'American Water Works Company Inc',
   'Ameriprise Financial',
   'AmerisourceBergen Corp',
   'AMETEK Inc.',
   'Amgen Inc.',
   'Amphenol Corp',
   'Analog Devices, Inc.',
   'ANSYS',
   'Anthem',
   'Ao

In [15]:
print(len(col[0][1]))
#print(col[0][1].pop())
#col[0][1].append('TransDigm Group')
print(len(col[0][1]))
print(len(col[1][1]))
print(len(col[2][1]))
print(len(col[3][1]))

443
443
443
443
443


In [16]:
#making the dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [17]:
#displaying the dataframe
df

,Company Name,CEO Name,CEO Pay,Median Employee Pay,CEO Pay Ratio
0,3M Company,Michael F. Roman,18321566,57494,319
1,Abbott Laboratories,na,-1,-1,-1
2,AbbVie Inc.,Richard A. Gonzalez,21610598,155885,139
3,ABIOMED Inc,Michael R. Minogue,16612766,112393,148
4,Accenture plc,na,-1,-1,-1
5,Activision Blizzard,Robert A. Kotick,30122896,94308,319
6,Adobe Inc.,Shantanu Narayen,39145631,147115,266
7,Advanced Micro Devices Inc,Lisa T. Su,58534288,96874,604
8,Advance Auto Parts,Thomas R. Greco,7691513,19463,395
9,AES Corp,na,-1,-1,-1


In [23]:
int = 0
for i in range(len(df)):
    if (df.at[i, 'Median Employee Pay'] > -1):
        int += 1
print(int)
print(443-int)

354
89


int = 0
for i in range(len(df)):
    if (df.at[i, 'Median Employee Pay'] == -1):
        int += 1
        print(i)
        print(df.at[i, 'Company Name'])
int

In [28]:
int = 0
for i in range(len(df)):
    if ('ney' in df.at[i, 'Company Name']):
        int += 1
        print(i)
int

230


1

In [29]:
df.at[230, 'Company Name']

"Honeywell Int'l Inc."

In [ ]:
companies.at[332, 'Security']

In [ ]:
companies1.at[332, 'Security']

In [ ]:
#finding incs

incs = []
int = 0
for i in range(len(companies)):
    if (df.at[i, 'Median Employee Pay'] == 0):
        int += 1
        incs.append(i)
print(int)
incs

In [31]:
df.to_csv(index=False)
compression_opts = dict(method='zip',
                        archive_name='ratios.csv')  
df.to_csv('ratios.csv', index=False,
          compression=compression_opts) 

TypeError: unhashable type: 'dict'